## Step-by-step guide to generating an interactive climate map in Folium (& Geopandas)

- **With some specific boilerplate code already filled in.**

---

<img src="../img/folium_logo.jpg" align="center" alt="bokeh_map" width="100/"/>

- **CREDITS**:
    
    - This tutorial was adapted by [Paul Wlodkowski](https://github.com/pawlodkowski) for the *Plotting on Maps* lesson @ Spiced Academy, based on original source code and lesson run by Malte Bonart, fellow Data Science Instructor @ Spiced Academy.
    
    
- **DATA**:

     - The data for this particular lesson is **synthetic** / **fake!** It was originally scraped from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/), but then regenerated based a simple polynomial trend and monthly seasonality modeled from the original data source & injected with random noise. Hence, the data in this repo is somewhat realistic but not authentic.


- **INSTALLATION**:

    - **Make sure you already have folium and geopandas installed**! (e.g. `pip` or `conda`), e.g.:

        - `pip install folium`

        - `pip install geopandas==0.8.0`
    

*Note*: this notebook was last run and tested on geopandas version 0.8.0 (released July 15, 2020). The latest version of folium should work just fine.

---

### STEP 1: Read in historical temperature data

In [ ]:
DATA = '../data/temp_data.csv'

In [ ]:
#Read in the data with pandas
import pandas as pd
df = pd.read_csv(DATA)

---
---

### STEP 2: Read in the geographic data (geometric shapes of all countries in the world) 
- Hint: Use GeoPandas
    - What is a **Shape file (.shp)?**
        - https://en.wikipedia.org/wiki/Shapefile#Shapefile_shape_format_(.shp)
- Publicly available GIS data downloaded from *Natural Earth*: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

In [ ]:
SHAPEFILE = '../data/ne_110m_admin_0_countries.shp'

In [ ]:
#Read in the shapefile with geopandas
import geopandas as gpd
gdf = gpd.read_file(____)

*What happens if I have trouble installing geopandas*?

- Unfortunately, this happened a couple times in the past with students, as the conda / pip installation on some Windows machines has been somewhat unreliable.
    - In this case, there have been some helpful posts from StackOverflow (for example, [this one](https://stackoverflow.com/questions/54734667/error-installing-geopandas-a-gdal-api-version-must-be-specified-in-anaconda)), which finally got the installation working.
- However, if you still have trouble with the installation despite lots of trial and error, you may consider creating a virtual environment or building a custom Docker image that builds off another image with Jupyter (e.g. https://hub.docker.com/r/jupyter/datascience-notebook)
    - If you choose to go this route, please reach out to a teacher for help. 
- If you would rather not try those options (for very understandable reasons), then don't worry; a teacher can provide you with a `.geojson` file you can use.
    - After all, the only reason we use GeoPandas in the first place is to read in the data and generate these GeoJSON files, so you shouldn't let this hold you back -- it's not that important in the end! 
    - Also, in general, with enough time spent googling around, you can find GeoJSON files from pretty much everywhere! For example:
        - https://github.com/johan/world.geo.json/tree/master/countries
        - https://github.com/isellsoap/deutschlandGeoJSON/blob/master/2_bundeslaender/4_niedrig.geo.json
        - https://github.com/funkeinteraktiv/Berlin-Geodaten/blob/master/berlin_bezirke.geojson
        - _And many more! They're super easy to find online! The world of open source is truly wonderful..._

---
---

### STEP 3: Group / aggregate the temperature anomaly data by country, year
- For simplicity, we're only interested in yearly averages

In [ ]:
df.groupby(...)

### STEP 4: Merge Data Sets.
- We want to have our temperature data and geometric data in one place.
- **Make sure you're still left with a GeoDataFrame at the end!**
    - otherwise, if the resulting object is a regular Pandas dataframe, the ``.to_json()`` export will produce a normal ``.json`` file rather than a specialized ``.geojson`` file.
    - A geojson file is very similar to a json file, with the exception that it's a bit more strict and specialized and works better with plotting libraries that usually expect the data to be in that format.

In [ ]:
gdf.merge(...)

---
---
### Time for Visualization with Folium

<img src="../img/folium_logo.jpg" align="left" alt="bokeh_logo" width="100/"/>

---
---

### STEP 5: Plot data on a map for a single year (we can make it interactive later)


   ### 5a. Generate a blank canvas / figure.

In [ ]:
import folium

In [ ]:
basemap = folium.Map(location=[52.54, 13.36],
                zoom_start=2,
                tiles='CartoDB positron') #try out the other options!

In [ ]:
basemap #display the map directly in Jupyter!



### 5a. Generate a GeoJSON string for a single year.
- The Folium library (as well as many other JavaScript-based mapping libraries) requires the data to be in GeoJSON format.
    - **HINT**: How can you convert a GeoDataFrame into a GeoJSON? Think of how you might do this in regular pandas.
- Let's use the year 2000 as an example.

In [ ]:
gdf_2000 = gdf[gdf['___'] == ___]
json_2000 = gdf_2000.to____()

### STEP 6: Generate an interactive choropleth map of the data for a single year
- Use the convenient ``folium.Choropleth()`` class to generate interactive tiles, which we can overlay on top of our base map!
- The trick to get it working is to make sure that the Choropleth layer understands which column from the dataframes is supposed to be mapped to which key in the JSON string in order for the data to render properly.

In [ ]:
tiles = folium.Choropleth(
            geo_data=____,                          # geojson string that includes the geo data (for the year 2000)
            name="____",
            data=____,                              # dataframe that includes the data (for the year 2000)
            columns=["country", "monthly_anomaly"], # names of the columns to include from the dataframe
            key_on="properties._____",              # name of JSON key within the "properties" value that contains country names
            fill_color="YlOrRd",                    # play around with the rest of the aesthetic options
            nan_fill_color="#ededed",         
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Monthly Temperature Anomaly",
            highlight=True,
        )

In [ ]:
tiles.add_to(basemap) #add the layer to the blank canvas / figure

In [ ]:
# basemap

### STEP 7: Add additional features to the map!
Some ideas:
- **Tooltip highlighting** (included here)
- Adding other layers / markers
- Making the tiles "clickable"
- Make the legend a fixed size
- The documentation isn't *great*, but with enough persistence / looking at examples online (some pretty good tutorials out there) / reading forums, you can add nice little widgets and extra interactivity to your map!

In [ ]:
style_html = "font-size: 10px; font-weight: normal" #add a little bit of HTML, if you know some.

tooltip = folium.features.GeoJsonTooltip(fields = ['country', 'monthly_anomaly'],
                                         style = style_html) 


In [ ]:
tiles.geojson.add_child(tooltip)
#overwrite some of the original properties of the geojson attribute of the tile

In [ ]:
basemap #the map should now display information whenever you hover over a country tile!

### STEP 8: Export the figure to an HTML file, so you can open it in your web browser!
- The reason the figure is interactive (e.g. you can zoom around, it has hover effects, etc.) is because there's a of front-end (i.e. client-side) JavaScript code that Plotly creates for you automatically.
- Use the ``.save()`` method to export the file and open it up in a web browser.
    - Bonus: Open the ``.html`` file in a text editor and see if you can understand any of it :)

In [ ]:
basemap.save("2000_map.html")

In [ ]:
#bash command to open the file (should use your web browser by default)
!open 2000_map.html

---

### And that's it!
- Unfortunately, there doesn't seem to be an easy way to add interactive sliders to the map (e.g. to change frames / years) like how you can do it in Plotly or Bokeh (at least, not as far as I could find).
    - Folium is pretty easy to use to make interactive maps quickly, but it seems to be lacking for more advanced features. The library is meant to be a bit more minimal / lightweight (which is also great, depending on your use case!)
    - You can, of course, write your own for-loop to generate multiple files for different years, but if you really want to add animation, you either have to write your own custom JavaScript (probably not a reasonable option) or switch over to a more powerful visualization library like Plotly or Bokeh.
    - However, there seem to be enough pretty good Folium tutorials / examples online where people have been able to make some pretty amazing things.